### Outline
- **Group based Apply**
    - `apply()`
    - Multi Indexing
- **Restructuring data** 
    - pd.melt()
    - pd.pivot()
    - pd.cut()
- **Dealing with Missing Values**
    - None and nan values
    - isna() and isnull()

- **String method in pandas**
    
- **Handling datetime**


- **Writing to a file** 


Let's first import our data and prepare it as we did in the last lecture

In [ ]:
import pandas as pd
import numpy as np
!gdown 1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
!gdown 1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm
movies = pd.read_csv('movies.csv', index_col=0)
directors = pd.read_csv('directors.csv',index_col=0)
data = movies.merge(directors, how='left', left_on='director_id',right_on='id')  
data.drop(['director_id','id_y'],axis=1,inplace=True)

Downloading...
From: https://drive.google.com/uc?id=1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
To: /content/movies.csv
100% 112k/112k [00:00<00:00, 49.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm
To: /content/directors.csv
100% 65.4k/65.4k [00:00<00:00, 47.6MB/s]


## Group based Apply


<!-- 
#### What if we want to do the transformation of a column using some column's agrregate

Lets say, we want to filter the movies whose budget was even higher than the average revenue of the director from his other movies -->
Now let's assume, we call a movies risky if,
- its budget is higher than the average revenue of its director 

#### How do we filter risky movies? 

We can subtract the average `revenue` of a director from `budget` col, for each director

Can use `transform` here?

**No**, since it uses only one column

#### Can you recall **another function like transform** which works with multiple columns?

In [ ]:
def func(x):
  x["risky"] = x["budget"] - x["revenue"].mean() >= 0
  return x
data_risky = data.groupby("director_name").apply(func)
data_risky

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,risky
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male,False
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male,False
2,43599,245.00,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male,False
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male,False
4,43602,258.00,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,Male,False
1461,48370,0.03,19,3.15,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,Male,False
1462,48375,0.00,7,0.00,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,Male,False
1463,48376,0.00,3,0.00,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,Male,False


Recall `apply()` from our earlier lectures

What did we do here?

- Defined a custom function
- Grouped data acc to `director_name`
- Subtracted mean of `budget` from `revenue` 
- Used apply with the custom function on the grouped data

Lets see if there are any risky movies

In [ ]:
data_risky.loc[data_risky["risky"]]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,risky
7,43608,200.0,107,586.09,Quantum of Solace,6.1,2965,2008,Oct,Thursday,Marc Forster,Male,True
12,43614,380.0,135,1045.71,Pirates of the Caribbean: On Stranger Tides,6.4,4948,2011,May,Saturday,Rob Marshall,Male,True
15,43618,200.0,37,310.67,Robin Hood,6.2,1398,2010,May,Wednesday,Ridley Scott,Male,True
20,43624,209.0,64,303.03,Battleship,5.5,2114,2012,Apr,Wednesday,Peter Berg,Male,True
24,43630,210.0,3,459.36,X-Men: The Last Stand,6.3,3525,2006,May,Wednesday,Brett Ratner,Male,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1347,47224,5.0,7,3.26,The Sweet Hereafter,6.8,103,1997,May,Wednesday,Atom Egoyan,Male,True
1349,47229,5.0,3,4.84,90 Minutes in Heaven,5.4,40,2015,Sep,Friday,Michael Polish,Male,True
1351,47233,5.0,6,0.00,Light Sleeper,5.7,15,1992,Aug,Friday,Paul Schrader,NaN,True
1356,47263,15.0,10,0.00,Dying of the Light,4.5,118,2014,Dec,Thursday,Paul Schrader,NaN,True


Yes, there are some 131 movies whose budget was **greater than average** earnings of its director

## Multi-Indexing

<!-- After **grouping the data**, select the required column and **apply an aggregate** on it -->

Now, lets say, you want to find who is the **most productive director**

#### Which director according to you would be considered as most productive ?

- Will you decide based on the **number of movies** released by a director?

Or 

- will consider **quality into consideration also?**


Or 

- will you also consider the amount of business the movie is doing? 



To simplify, 

Lets calculate who has directed maximum number of movies

In [ ]:
data.groupby(['director_name'])['title'].count().sort_values(ascending=False)

director_name
Steven Spielberg    26
Clint Eastwood      19
Martin Scorsese     19
Woody Allen         18
Robert Rodriguez    16
                    ..
Paul Weitz           5
John Madden          5
Paul Verhoeven       5
John Whitesell       5
Kevin Reynolds       5
Name: title, Length: 199, dtype: int64

Looks like `Steven Spielberg` has directed maximum number of movies

#### But does it make `Steven` the most productive director?

Chances are, he might be **active for more years** than other directors

#### How would you calculate active years for **every director**?

We can subtract both `min` and `max` of `year`

#### How can we calculate multiple aggregates such as `min` and `max`, along with count of `titles` together?

In [ ]:
data_agg = data.groupby(['director_name'])[["year", "title"]].aggregate({"year":['min','max'], "title": "count"})
data_agg


year       title
                              min   max count
director_name                                
Adam McKay                   2004  2015     6
Adam Shankman                2001  2012     8
Alejandro González Iñárritu  2000  2015     6
Alex Proyas                  1994  2016     5
Alexander Payne              1999  2013     5
...                           ...   ...   ...
Wes Craven                   1984  2011    10
Wolfgang Petersen            1981  2006     7
Woody Allen                  1977  2013    18
Zack Snyder                  2004  2016     7
Zhang Yimou                  2002  2014     6

[199 rows x 3 columns]

Notice,
- `director_name` column has turned into **row labels**
- There are multiple levels for the column names

This is called **Multi-index Dataframe**

#### What is Multi-index Dataframe ?
  - It can have **multiple indexes along a dimension**
    - no of dimensions remain same though => 2D
  - Multi-level indexes are **possible both for rows and columns**


In [ ]:
data_agg.columns #Printing the columns for better clarity

MultiIndex([( 'year',   'min'),
            ( 'year',   'max'),
            ('title', 'count')],
           )

The level-1 column names are `year` and `title`

#### What would happen if we print the col `year` of this multi-index dataframe?

In [ ]:
data_agg["year"]

,min,max
director_name,,
Adam McKay,2004,2015
Adam Shankman,2001,2012
Alejandro González Iñárritu,2000,2015
Alex Proyas,1994,2016
Alexander Payne,1999,2013
...,...,...
Wes Craven,1984,2011
Wolfgang Petersen,1981,2006
Woody Allen,1977,2013


#### How can we convert multi-level back to only one level of columns?

Example: `year_min`, `year_max`, `title_count`

In [ ]:
data_agg.columns = ['_'.join(col) for col in data_agg.columns]
data_agg

,year_min,year_max,title_count
director_name,,,
Adam McKay,2004,2015,6
Adam Shankman,2001,2012,8
Alejandro González Iñárritu,2000,2015,6
Alex Proyas,1994,2016,5
Alexander Payne,1999,2013,5
...,...,...,...
Wes Craven,1984,2011,10
Wolfgang Petersen,1981,2006,7
Woody Allen,1977,2013,18


Since these were tuples, we can just join them

In [ ]:
data.groupby('director_name')[['year', 'title']].agg(
    year_max=('year','max'),
    year_min=('year','min'),
    title_count=('title','count')
)


,year_max,year_min,title_count
director_name,,,
Adam McKay,2015,2004,6
Adam Shankman,2012,2001,8
Alejandro González Iñárritu,2015,2000,6
Alex Proyas,2016,1994,5
Alexander Payne,2013,1999,5
...,...,...,...
Wes Craven,2011,1984,10
Wolfgang Petersen,2006,1981,7
Woody Allen,2013,1977,18


Columns look good, but we may want to turn back the row labels into a proper column as well

#### How can we convert row labels into a column?

In [ ]:
data_agg.reset_index()

,director_name,year_min,year_max,title_count
0,Adam McKay,2004,2015,6
1,Adam Shankman,2001,2012,8
2,Alejandro González Iñárritu,2000,2015,6
3,Alex Proyas,1994,2016,5
4,Alexander Payne,1999,2013,5
...,...,...,...,...
194,Wes Craven,1984,2011,10
195,Wolfgang Petersen,1981,2006,7
196,Woody Allen,1977,2013,18
197,Zack Snyder,2004,2016,7



Recall, 

We learnt `reset_index()` earlier

#### Using the new features, can we find the most productive director?

First calculate how many years the director has been active.



In [ ]:
data_agg["yrs_active"] = data_agg["year_max"] - data_agg["year_min"]
data_agg

,year_min,year_max,title_count,yrs_active
director_name,,,,
Adam McKay,2004,2015,6,11
Adam Shankman,2001,2012,8,11
Alejandro González Iñárritu,2000,2015,6,15
Alex Proyas,1994,2016,5,22
Alexander Payne,1999,2013,5,14
...,...,...,...,...
Wes Craven,1984,2011,10,27
Wolfgang Petersen,1981,2006,7,25
Woody Allen,1977,2013,18,36


Then calculate rate of directing movies by `title_count`/`yrs_active`

In [ ]:
data_agg["movie_per_yr"] = data_agg["title_count"] / data_agg["yrs_active"]
data_agg

,year_min,year_max,title_count,yrs_active,movie_per_yr
director_name,,,,,
Adam McKay,2004,2015,6,11,0.545455
Adam Shankman,2001,2012,8,11,0.727273
Alejandro González Iñárritu,2000,2015,6,15,0.400000
Alex Proyas,1994,2016,5,22,0.227273
Alexander Payne,1999,2013,5,14,0.357143
...,...,...,...,...,...
Wes Craven,1984,2011,10,27,0.370370
Wolfgang Petersen,1981,2006,7,25,0.280000
Woody Allen,1977,2013,18,36,0.500000


Now finally sort the values

In [ ]:
data_agg.sort_values("movie_per_yr", ascending=False)

,year_min,year_max,title_count,yrs_active,movie_per_yr
director_name,,,,,
Tyler Perry,2006,2013,9,7,1.285714
Jason Friedberg,2006,2010,5,4,1.250000
Shawn Levy,2002,2014,11,12,0.916667
Robert Rodriguez,1992,2014,16,22,0.727273
Adam Shankman,2001,2012,8,11,0.727273
...,...,...,...,...,...
Lawrence Kasdan,1985,2012,5,27,0.185185
Luc Besson,1985,2014,5,29,0.172414
Robert Redford,1980,2010,5,30,0.166667


**Conclusion:**

==> "Tyler Perry" turns out to be the **truly most productive director**




## Pfizer Usecase Agenda

- We'll continue looking at **Pandas** library

- We'll cover **Restructuring Data using Pandas**

- We'll also look at **handling missing values** and **understanding DateTime datatype**



## Importing our data

- For this topic we will be using **data of few drugs** being developed by **PFizer**


Link: https://drive.google.com/file/d/173A59xh2mnpmljCCB9bhC4C5eP2IS6qZ/view?usp=sharing

In [ ]:
!gdown 173A59xh2mnpmljCCB9bhC4C5eP2IS6qZ

Downloading...
From: https://drive.google.com/uc?id=173A59xh2mnpmljCCB9bhC4C5eP2IS6qZ
To: /content/Pfizer_1.csv
100% 1.51k/1.51k [00:00<00:00, 2.50MB/s]


#### What is the data about?

- Temperature (K)
- Pressure (P)

are recorded after an **interval of 1 hour** everyday to monitor the drug stability in a drug development test

==> These data points are thus used to **identify the optimal set of values of parameters** for the stability of the drugs

#### Now, Let's explore this dataset


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('Pfizer_1.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       18 non-null     object 
 1   Drug_Name  18 non-null     object 
 2   Parameter  18 non-null     object 
 3   1:30:00    16 non-null     float64
 4   2:30:00    16 non-null     float64
 5   3:30:00    12 non-null     float64
 6   4:30:00    14 non-null     float64
 7   5:30:00    16 non-null     float64
 8   6:30:00    18 non-null     int64  
 9   7:30:00    16 non-null     float64
 10  8:30:00    14 non-null     float64
 11  9:30:00    16 non-null     float64
 12  10:30:00   18 non-null     int64  
 13  11:30:00   16 non-null     float64
 14  12:30:00   18 non-null     int64  
dtypes: float64(9), int64(3), object(3)
memory usage: 2.2+ KB


In [ ]:
data.head()

,Date,Drug_Name,Parameter,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00,10:30:00,11:30:00,12:30:00
0,15-10-2020,diltiazem hydrochloride,Temperature,23.0,22.0,NaN,21.0,21.0,22,23.0,21.0,22.0,20,20.0,21
1,15-10-2020,diltiazem hydrochloride,Pressure,12.0,13.0,NaN,11.0,13.0,14,16.0,16.0,24.0,18,19.0,20
2,15-10-2020,docetaxel injection,Temperature,NaN,17.0,18.0,NaN,17.0,18,NaN,NaN,23.0,23,25.0,25
3,15-10-2020,docetaxel injection,Pressure,NaN,22.0,22.0,NaN,22.0,23,NaN,NaN,27.0,26,29.0,28
4,15-10-2020,ketamine hydrochloride,Temperature,24.0,NaN,NaN,27.0,NaN,26,25.0,24.0,23.0,22,21.0,20


In [ ]:
data.tail()

,Date,Drug_Name,Parameter,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00,10:30:00,11:30:00,12:30:00
13,17-10-2020,diltiazem hydrochloride,Pressure,3.0,4.0,4.0,4.0,6.0,8,9.0,NaN,9.0,11,13.0,14
14,17-10-2020,docetaxel injection,Temperature,12.0,13.0,14.0,15.0,16.0,17,18.0,19.0,20.0,21,22.0,23
15,17-10-2020,docetaxel injection,Pressure,20.0,22.0,22.0,22.0,22.0,23,25.0,26.0,27.0,28,29.0,28
16,17-10-2020,ketamine hydrochloride,Temperature,13.0,14.0,15.0,16.0,17.0,18,19.0,20.0,21.0,22,23.0,24
17,17-10-2020,ketamine hydrochloride,Pressure,8.0,9.0,10.0,11.0,11.0,12,12.0,11.0,12.0,13,14.0,15


<!-- #### Let's check the shape of this dataset -->

## Melting in Pandas

As we saw earlier, the dataset has 18 rows and 15 columns

If you notice further, you'll see:

- The **columns are `1:30:00`, `2:30:00`, `3:30:00`, ... so on**

- `Temperature` and `Pressure` **of each date** is **in a separate row**

#### Can we restructure our data into a better format?

<!-- Maybe do something more intuitive -->

Maybe we can have a column for `time`, with `timestamps` as the column value

**Where will the Temperature/Pressure values go?**

We can similarly create one column containing the values of these parameters




==> **"Melt" timestamp columns into two columns** - timestamp and corresponding values
<!-- Something like our DataFrame will have columns [`ID`, `Date`, `Parameter`, `time`, `result`] -->




#### How can we restructure our data into having every row corresponding to a single reading?


In [ ]:
pd.melt(data, id_vars=['Date', 'Parameter', 'Drug_Name'])                

,Date,Parameter,Drug_Name,variable,value
0,15-10-2020,Temperature,diltiazem hydrochloride,1:30:00,23.0
1,15-10-2020,Pressure,diltiazem hydrochloride,1:30:00,12.0
2,15-10-2020,Temperature,docetaxel injection,1:30:00,NaN
3,15-10-2020,Pressure,docetaxel injection,1:30:00,NaN
4,15-10-2020,Temperature,ketamine hydrochloride,1:30:00,24.0
...,...,...,...,...,...
211,17-10-2020,Pressure,diltiazem hydrochloride,12:30:00,14.0
212,17-10-2020,Temperature,docetaxel injection,12:30:00,23.0
213,17-10-2020,Pressure,docetaxel injection,12:30:00,28.0
214,17-10-2020,Temperature,ketamine hydrochloride,12:30:00,24.0


This converts our data from `wide` to `long` format


Notice the `id_vars are set of variables which remain unmelted

#### How does `pd.melt()` work?


- Pass in the **DataFrame**
- Pass in the **column names to not melt** <!--**that we DON'T want to change** -->




But we can provide better names to these new columns

#### How can we rename the columns "variable" and "value" as per our original dataframe?

In [ ]:
data_melt = pd.melt(data,id_vars = ['Date', 'Drug_Name', 'Parameter'], 
            var_name = "time",      
            value_name = 'reading')  

data_melt

,Date,Drug_Name,Parameter,time,reading
0,15-10-2020,diltiazem hydrochloride,Temperature,1:30:00,23.0
1,15-10-2020,diltiazem hydrochloride,Pressure,1:30:00,12.0
2,15-10-2020,docetaxel injection,Temperature,1:30:00,NaN
3,15-10-2020,docetaxel injection,Pressure,1:30:00,NaN
4,15-10-2020,ketamine hydrochloride,Temperature,1:30:00,24.0
...,...,...,...,...,...
211,17-10-2020,diltiazem hydrochloride,Pressure,12:30:00,14.0
212,17-10-2020,docetaxel injection,Temperature,12:30:00,23.0
213,17-10-2020,docetaxel injection,Pressure,12:30:00,28.0
214,17-10-2020,ketamine hydrochloride,Temperature,12:30:00,24.0


**Conclusion**

<!-- - Columns from `1:30:00` to `12:30:00` are conviniently **melted to a single column `time`** -->
- The labels of the timestamp columns are conviniently **melted into a single column** - `time`
- It retained all values in column `reading`

--------------

- The labels of columns such as `1:30:00`, `2:30:00` have now become categories of the variable column
- The **values from columns we are melting** are stored in **value** column


## Pivot

Now suppose we want to convert our data back to **wide format**

The reason could be to maintain the structure for storing or some other purpose.

Notice:

- The variables `Date`, `Drug_Name` and `Parameter` will remain same

- The column names will be extracted from the column `time`

- The values will be extracted from the column `readings`

#### How can we restructure our data back to the original wide format, before it was melted?




In [ ]:
data_melt.pivot(index=['Date','Drug_Name','Parameter'],  # Column to use to make new frame’s index
                columns = 'time',                  # Column to use to make new frame’s columns
                values='reading')                   # Columns to use for populating new frame’s values.

time                                            10:30:00  11:30:00  12:30:00  \
Date       Drug_Name               Parameter                                   
15-10-2020 diltiazem hydrochloride Pressure         18.0      19.0      20.0   
                                   Temperature      20.0      20.0      21.0   
           docetaxel injection     Pressure         26.0      29.0      28.0   
                                   Temperature      23.0      25.0      25.0   
           ketamine hydrochloride  Pressure          9.0       9.0      11.0   
                                   Temperature      22.0      21.0      20.0   
16-10-2020 diltiazem hydrochloride Pressure         24.0       NaN      27.0   
                                   Temperature      40.0       NaN      42.0   
           docetaxel injection     Pressure         28.0      29.0      30.0   
                                   Temperature      56.0      57.0      58.0   
           ketamine hydrochloride  Pressure         16.0      17.0      18.0   
                                   Temperature      13.0      14.0      15.0   
17-10-2020 diltiazem hydrochloride Pressure         11.0      13.0      14.0   
                                   Temperature      14.0      11.0      10.0   
           docetaxel injection     Pressure         28.0      29.0      28.0   
                                   Temperature      21.0      22.0      23.0   
           ketamine hydrochloride  Pressure         13.0      14.0      15.0   
                                   Temperature      22.0      23.0      24.0   

time                                            1:30:00  2:30:00  3:30:00  \
Date       Drug_Name               Parameter                                
15-10-2020 diltiazem hydrochloride Pressure        12.0     13.0      NaN   
                                   Temperature     23.0     22.0      NaN   
           docetaxel injection     Pressure         NaN     22.0     22.0   
                                   Temperature      NaN     17.0     18.0   
           ketamine hydrochloride  Pressure         8.0      NaN      NaN   
                                   Temperature     24.0      NaN      NaN   
16-10-2020 diltiazem hydrochloride Pressure        18.0     19.0     20.0   
                                   Temperature     34.0     35.0     36.0   
           docetaxel injection     Pressure        23.0     24.0      NaN   
                                   Temperature     46.0     47.0      NaN   
           ketamine hydrochloride  Pressure        12.0     12.0     13.0   
                                   Temperature      8.0      9.0     10.0   
17-10-2020 diltiazem hydrochloride Pressure         3.0      4.0      4.0   
                                   Temperature     20.0     19.0     19.0   
           docetaxel injection     Pressure        20.0     22.0     22.0   
                                   Temperature     12.0     13.0     14.0   
           ketamine hydrochloride  Pressure         8.0      9.0     10.0   
                                   Temperature     13.0     14.0     15.0   

time                                            4:30:00  5:30:00  6:30:00  \
Date       Drug_Name               Parameter                                
15-10-2020 diltiazem hydrochloride Pressure        11.0     13.0     14.0   
                                   Temperature     21.0     21.0     22.0   
           docetaxel injection     Pressure         NaN     22.0     23.0   
                                   Temperature      NaN     17.0     18.0   
           ketamine hydrochloride  Pressure         7.0      NaN      9.0   
                                   Temperature     27.0      NaN     26.0   
16-10-2020 diltiazem hydrochloride Pressure        21.0     22.0     23.0   
                                   Temperature     36.0     37.0     38.0   
           docetaxel injection     Pressure        25.0     26.0     27.0   
           

Notice, 

We are getting **multiple indices** here

#### How can we reset this to a single-index dataframe?

In [ ]:
data_melt.pivot(index=['Date','Drug_Name','Parameter'], 
                columns = 'time',                  
                values='reading').reset_index()

time,Date,Drug_Name,Parameter,10:30:00,11:30:00,12:30:00,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00
0,15-10-2020,diltiazem hydrochloride,Pressure,18.0,19.0,20.0,12.0,13.0,NaN,11.0,13.0,14.0,16.0,16.0,24.0
1,15-10-2020,diltiazem hydrochloride,Temperature,20.0,20.0,21.0,23.0,22.0,NaN,21.0,21.0,22.0,23.0,21.0,22.0
2,15-10-2020,docetaxel injection,Pressure,26.0,29.0,28.0,NaN,22.0,22.0,NaN,22.0,23.0,NaN,NaN,27.0
3,15-10-2020,docetaxel injection,Temperature,23.0,25.0,25.0,NaN,17.0,18.0,NaN,17.0,18.0,NaN,NaN,23.0
4,15-10-2020,ketamine hydrochloride,Pressure,9.0,9.0,11.0,8.0,NaN,NaN,7.0,NaN,9.0,10.0,11.0,10.0
5,15-10-2020,ketamine hydrochloride,Temperature,22.0,21.0,20.0,24.0,NaN,NaN,27.0,NaN,26.0,25.0,24.0,23.0
6,16-10-2020,diltiazem hydrochloride,Pressure,24.0,NaN,27.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,25.0
7,16-10-2020,diltiazem hydrochloride,Temperature,40.0,NaN,42.0,34.0,35.0,36.0,36.0,37.0,38.0,37.0,38.0,39.0
8,16-10-2020,docetaxel injection,Pressure,28.0,29.0,30.0,23.0,24.0,NaN,25.0,26.0,27.0,28.0,29.0,28.0
9,16-10-2020,docetaxel injection,Temperature,56.0,57.0,58.0,46.0,47.0,NaN,48.0,48.0,49.0,50.0,52.0,55.0



==> `pivot()` is the exact opposite of melt

#### How does `pivot()` work?

- Column `Time` is pivoted upon `Date`, `Drug_Name` and `Parameter`





In [ ]:
data_melt.head()

,Date,Drug_Name,Parameter,time,reading
0,15-10-2020,diltiazem hydrochloride,Temperature,1:30:00,23.0
1,15-10-2020,diltiazem hydrochloride,Pressure,1:30:00,12.0
2,15-10-2020,docetaxel injection,Temperature,1:30:00,NaN
3,15-10-2020,docetaxel injection,Pressure,1:30:00,NaN
4,15-10-2020,ketamine hydrochloride,Temperature,1:30:00,24.0


Now if you notice,

We are **using 2 rows** to log readings for a single experiment. 


#### Can we further restructure our data into dividing the Parameter column into T/P?

A format like:

`Date | time | Drug_Name | Pressure | Temperature`

 would be really suitable

- We want to **split one single column into multiple columns**

#### How can we divide the Parameter column again?

In [ ]:
data_tidy = data_melt.pivot(index=['Date','time', 'Drug_Name'], 
                                        columns = 'Parameter',  
                                        values='reading') 

data_tidy

Parameter                                    Pressure  Temperature
Date       time     Drug_Name                                     
15-10-2020 10:30:00 diltiazem hydrochloride      18.0         20.0
                    docetaxel injection          26.0         23.0
                    ketamine hydrochloride        9.0         22.0
           11:30:00 diltiazem hydrochloride      19.0         20.0
                    docetaxel injection          29.0         25.0
...                                               ...          ...
17-10-2020 8:30:00  docetaxel injection          26.0         19.0
                    ketamine hydrochloride       11.0         20.0
           9:30:00  diltiazem hydrochloride       9.0         13.0
                    docetaxel injection          27.0         20.0
                    ketamine hydrochloride       12.0         21.0

[108 rows x 2 columns]

<!-- Notice that a **multi-index** df has been created

We change this using **`reset_index()`** -->

We can use `reset_index()` to remove the multi-index

In [ ]:
data_tidy = data_tidy.reset_index()
data_tidy

Parameter,Date,time,Drug_Name,Pressure,Temperature
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0
...,...,...,...,...,...
103,17-10-2020,8:30:00,docetaxel injection,26.0,19.0
104,17-10-2020,8:30:00,ketamine hydrochloride,11.0,20.0
105,17-10-2020,9:30:00,diltiazem hydrochloride,9.0,13.0
106,17-10-2020,9:30:00,docetaxel injection,27.0,20.0


We can rename our ```index``` column from `Parameter` to simply `None`

In [ ]:
data_tidy.columns.name = 'None'

In [ ]:
data_tidy.head()

None,Date,time,Drug_Name,Pressure,Temperature
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0


## Handling Missing Values



If you notice, there are many "NaN" values in our data

In [ ]:
data_tidy.head()

None,Date,time,Drug_Name,Pressure,Temperature
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0


#### What are these "NaN" values?
They are basically **missing values**

#### What are missing values?
A Missing Value signifies an **empty cell/no data**

There can be 2 kinds of missing values:

  1. `None`
  2. `NaN` (short for Not a Number)

#### Whats the difference between the "None" and "NaN"?

The diff mainly lies in their datatype

In [ ]:
type(None)

NoneType

In [ ]:
type(np.nan)

float

**None type** is for missing values in a column with **non-number entries** 
- E.g.-strings

**NaN** occurs for columns with **number entries**

Note:

Pandas uses these values nearly **interchangeably**, converting between them where appropriate, based on column datatype

In [ ]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For **numerical** types, Pandas changes **None to NaN** type


In [ ]:
pd.Series(["1", "np.nan", "2", None])

0         1
1    np.nan
2         2
3      None
dtype: object

In [ ]:
pd.Series(["1", "np.nan", "2", np.nan])

0         1
1    np.nan
2         2
3       NaN
dtype: object

For **object** type, the **None is preserved** and not changed to NaN

Now we have the basic idea about missing values

#### How to know the count of missing values for each row/column?


In [ ]:
data.isna().head()

,Date,Drug_Name,Parameter,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00,10:30:00,11:30:00,12:30:00
0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,True,False,False,True,True,False,False,False,False
3,False,False,False,True,False,False,True,False,False,True,True,False,False,False,False
4,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False


We can also use isnull to get the same results

In [ ]:
data.isnull().head()

,Date,Drug_Name,Parameter,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00,10:30:00,11:30:00,12:30:00
0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,True,False,False,True,True,False,False,False,False
3,False,False,False,True,False,False,True,False,False,True,True,False,False,False,False
4,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False


#### But, why do we have two methods, "isna" and "isnull" for the same operation?

isnull() is just an alias for isna()

In [ ]:
pd.isnull

<function pandas.core.dtypes.missing.isna(obj)>

In [ ]:
pd.isna

<function pandas.core.dtypes.missing.isna(obj)>

As we can see, function signature is same for both

`isna()` returns a **boolean dataframe**, with each cell as a boolean value

This value corresponds to **whether the cell has a missing value**

On top of this, we can use `.sum()` to find the count

In [ ]:
data.isna().sum()

Date         0
Drug_Name    0
Parameter    0
1:30:00      2
2:30:00      2
3:30:00      6
4:30:00      4
5:30:00      2
6:30:00      0
7:30:00      2
8:30:00      4
9:30:00      2
10:30:00     0
11:30:00     2
12:30:00     0
dtype: int64

This gives us the total number of missing values in each column

#### Can we also get the number of missing values in each row?

In [ ]:
data.isna().sum(axis=1)

0     1
1     1
2     4
3     4
4     3
5     3
6     1
7     1
8     1
9     1
10    2
11    2
12    1
13    1
14    0
15    0
16    0
17    0
dtype: int64

Note:

By default the value is `axis=0` in sum()


#### We have identified the null count, but how do we deal with them?

We have two options:
- delete the rows/columns containing the null values
- fill the missing values with some data/estimate

Let's first look at deleting the rows

#### How can we drop rows containing null values?

In [ ]:
data.dropna()

,Date,Drug_Name,Parameter,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00,10:30:00,11:30:00,12:30:00
14,17-10-2020,docetaxel injection,Temperature,12.0,13.0,14.0,15.0,16.0,17,18.0,19.0,20.0,21,22.0,23
15,17-10-2020,docetaxel injection,Pressure,20.0,22.0,22.0,22.0,22.0,23,25.0,26.0,27.0,28,29.0,28
16,17-10-2020,ketamine hydrochloride,Temperature,13.0,14.0,15.0,16.0,17.0,18,19.0,20.0,21.0,22,23.0,24
17,17-10-2020,ketamine hydrochloride,Pressure,8.0,9.0,10.0,11.0,11.0,12,12.0,11.0,12.0,13,14.0,15


Rows with **even a single missing value** have been deleted

#### What if we want to delete the columns having missing value?


In [ ]:
data.dropna(axis=1)

,Date,Drug_Name,Parameter,6:30:00,10:30:00,12:30:00
0,15-10-2020,diltiazem hydrochloride,Temperature,22,20,21
1,15-10-2020,diltiazem hydrochloride,Pressure,14,18,20
2,15-10-2020,docetaxel injection,Temperature,18,23,25
3,15-10-2020,docetaxel injection,Pressure,23,26,28
4,15-10-2020,ketamine hydrochloride,Temperature,26,22,20
5,15-10-2020,ketamine hydrochloride,Pressure,9,9,11
6,16-10-2020,diltiazem hydrochloride,Temperature,38,40,42
7,16-10-2020,diltiazem hydrochloride,Pressure,23,24,27
8,16-10-2020,docetaxel injection,Temperature,49,56,58
9,16-10-2020,docetaxel injection,Pressure,27,28,30


=> Every column which had even a single missing value has been deleted

#### But what are the problems with deleting rows/columns?

One of the major problems: 
- loss of data

Instead of dropping, it would be better to **fill the missing values with some data**

#### How can we fill the missing values with some data?

In [ ]:
data.fillna(0).head()

,Date,Drug_Name,Parameter,1:30:00,2:30:00,3:30:00,4:30:00,5:30:00,6:30:00,7:30:00,8:30:00,9:30:00,10:30:00,11:30:00,12:30:00
0,15-10-2020,diltiazem hydrochloride,Temperature,23.0,22.0,0.0,21.0,21.0,22,23.0,21.0,22.0,20,20.0,21
1,15-10-2020,diltiazem hydrochloride,Pressure,12.0,13.0,0.0,11.0,13.0,14,16.0,16.0,24.0,18,19.0,20
2,15-10-2020,docetaxel injection,Temperature,0.0,17.0,18.0,0.0,17.0,18,0.0,0.0,23.0,23,25.0,25
3,15-10-2020,docetaxel injection,Pressure,0.0,22.0,22.0,0.0,22.0,23,0.0,0.0,27.0,26,29.0,28
4,15-10-2020,ketamine hydrochloride,Temperature,24.0,0.0,0.0,27.0,0.0,26,25.0,24.0,23.0,22,21.0,20


**What is fillna(0) doing?**

It fills all missing values with 0

We can do the same on a particular column too

In [ ]:
data['2:30:00'].fillna(0)

0     22.0
1     13.0
2     17.0
3     22.0
4      0.0
5      0.0
6     35.0
7     19.0
8     47.0
9     24.0
10     9.0
11    12.0
12    19.0
13     4.0
14    13.0
15    22.0
16    14.0
17     9.0
Name: 2:30:00, dtype: float64


#### What other values can we use to fill the missing values ?

We can use some **kind of estimator** too
- An estimator like **mean or median**

#### How would you calculate the mean of the column `2:30:00`?


In [ ]:
data['2:30:00'].mean()

18.8125

Now let's fill the NaN values with the mean value of the column

In [ ]:
data['2:30:00'].fillna(data['2:30:00'].mean())

0     22.0000
1     13.0000
2     17.0000
3     22.0000
4     18.8125
5     18.8125
6     35.0000
7     19.0000
8     47.0000
9     24.0000
10     9.0000
11    12.0000
12    19.0000
13     4.0000
14    13.0000
15    22.0000
16    14.0000
17     9.0000
Name: 2:30:00, dtype: float64

But this doesn't feel right. What could be wrong with this?

#### Can we use the mean of all compounds as average for our estimator?

- **Different drugs** have **different characteristics** 
- We can't simply do an average and fill the null values

**Then what could be a solution here?**

We could fill the null values of **respective compounds with their respective means**






#### How can we form a column with mean temperature of respective compounds?

We can use `apply` that we learnt earlier

Let's first create a function to calculate the mean

In [ ]:
def temp_mean(x):
  x['Temperature_avg'] = x['Temperature'].mean() # We will name the new col Temperature_avg
  return x

Now we can form a new column based on the average values of temperature for each drug

In [ ]:
data_tidy=data_tidy.groupby(["Drug_Name"]).apply(temp_mean)
data_tidy

None,Date,time,Drug_Name,Pressure,Temperature,Temperature_avg
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0,24.848485
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0,30.387097
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0,17.709677
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0,24.848485
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0,30.387097
...,...,...,...,...,...,...
103,17-10-2020,8:30:00,docetaxel injection,26.0,19.0,30.387097
104,17-10-2020,8:30:00,ketamine hydrochloride,11.0,20.0,17.709677
105,17-10-2020,9:30:00,diltiazem hydrochloride,9.0,13.0,24.848485
106,17-10-2020,9:30:00,docetaxel injection,27.0,20.0,30.387097


Now we fill the null values in Temperature using this new column!

In [ ]:
data_tidy['Temperature'].fillna(data_tidy["Temperature_avg"], inplace=True)
data_tidy

None,Date,time,Drug_Name,Pressure,Temperature,Temperature_avg
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0,24.848485
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0,30.387097
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0,17.709677
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0,24.848485
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0,30.387097
...,...,...,...,...,...,...
103,17-10-2020,8:30:00,docetaxel injection,26.0,19.0,30.387097
104,17-10-2020,8:30:00,ketamine hydrochloride,11.0,20.0,17.709677
105,17-10-2020,9:30:00,diltiazem hydrochloride,9.0,13.0,24.848485
106,17-10-2020,9:30:00,docetaxel injection,27.0,20.0,30.387097


In [ ]:
data_tidy.isna().sum()

None
Date                0
time                0
Drug_Name           0
Pressure           13
Temperature         0
Temperature_avg     0
dtype: int64

Great!! 

We have removed the null values of our Temperature column

Let's do the same for Pressure

In [ ]:
def pr_mean(x):
  x['Pressure_avg'] = x['Pressure'].mean()
  return x
data_tidy=data_tidy.groupby(["Drug_Name"]).apply(pr_mean)
data_tidy['Pressure'].fillna(data_tidy["Pressure_avg"], inplace=True)
data_tidy

None,Date,time,Drug_Name,Pressure,Temperature,Temperature_avg,Pressure_avg
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0,24.848485,15.424242
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0,30.387097,25.483871
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0,17.709677,11.935484
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0,24.848485,15.424242
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0,30.387097,25.483871
...,...,...,...,...,...,...,...
103,17-10-2020,8:30:00,docetaxel injection,26.0,19.0,30.387097,25.483871
104,17-10-2020,8:30:00,ketamine hydrochloride,11.0,20.0,17.709677,11.935484
105,17-10-2020,9:30:00,diltiazem hydrochloride,9.0,13.0,24.848485,15.424242
106,17-10-2020,9:30:00,docetaxel injection,27.0,20.0,30.387097,25.483871


In [ ]:
data_tidy.isna().sum()

None
Date               0
time               0
Drug_Name          0
Pressure           0
Temperature        0
Temperature_avg    0
Pressure_avg       0
dtype: int64

This gives us a **basic idea** about working with missing values

We will further learn more on this during later lectures of **feature engineering**

## Pandas Cut




Sometimes, we would want our data to be in **categorical format instead of continous data**. 

#### What do we mean by converting continous into categorical data?

Lets say, instead of knowing specific test values of a month, I want to know its type

#### What could be the types?

Depends on level of granularity we want to have - Low, Medium, High, V High

We could have defined more (or less) categories

#### But how can bucketisation of continous data help?

- Since, we can get the count of different categories
- We can get a idea of the bin which category (range of values) most of the temperature values lie.

#### What function can we use to convert cont. to cat. data?

 - Will use pd.cut()
 - We need to provide:
  - the continous data
  - bins edges (array of numbers) to "cut" the entire range
  - labels corresponding to every bin


Let's try to us this on our max (temp) column to categorise the data into bins

But, to define categories, lets first check min and max temp values 

In [ ]:
data_tidy

None,Date,time,Drug_Name,Pressure,Temperature,Temperature_avg,Pressure_avg
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0,24.848485,15.424242
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0,30.387097,25.483871
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0,17.709677,11.935484
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0,24.848485,15.424242
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0,30.387097,25.483871
...,...,...,...,...,...,...,...
103,17-10-2020,8:30:00,docetaxel injection,26.0,19.0,30.387097,25.483871
104,17-10-2020,8:30:00,ketamine hydrochloride,11.0,20.0,17.709677,11.935484
105,17-10-2020,9:30:00,diltiazem hydrochloride,9.0,13.0,24.848485,15.424242
106,17-10-2020,9:30:00,docetaxel injection,27.0,20.0,30.387097,25.483871


In [ ]:
print(data_tidy['Temperature'].min(), data_tidy['Temperature'].max())

8.0 58.0


Min value = 8, Max value is 58.

- Lets's keep some buffer for future values and take the range from 5-60(instead of 8-58)
- Lets divide this data into 4 bins of 10-15 values each

In [ ]:
temp_points = [5, 20, 35, 50, 60]
temp_labels = ['low','medium','high','very_high'] # Here labels define the severity of the resultant output of the test
data_tidy['temp_cat'] = pd.cut(data_tidy['Temperature'], bins=temp_points, labels=temp_labels)
data_tidy.head()

None,Date,time,Drug_Name,Pressure,Temperature,Temperature_avg,Pressure_avg,temp_cat
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0,24.848485,15.424242,low
1,15-10-2020,10:30:00,docetaxel injection,26.0,23.0,30.387097,25.483871,medium
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0,17.709677,11.935484,medium
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0,24.848485,15.424242,low
4,15-10-2020,11:30:00,docetaxel injection,29.0,25.0,30.387097,25.483871,medium


In [ ]:
data_tidy['temp_cat'].value_counts()

low          50
medium       38
high         15
very_high     5
Name: temp_cat, dtype: int64

## String functions


 

#### What kind of questions can we use string methods for?

Find rows which contains a particular string

Say,

####How you can you filter rows containing "hydrochloric" in their drug name?


In [ ]:
data_tidy.loc[data_tidy['Drug_Name'].str.contains('hydrochloride')].head()

None,Date,time,Drug_Name,Pressure,Temperature,Temperature_avg,Pressure_avg,temp_cat
0,15-10-2020,10:30:00,diltiazem hydrochloride,18.0,20.0,24.848485,15.424242,low
2,15-10-2020,10:30:00,ketamine hydrochloride,9.0,22.0,17.709677,11.935484,medium
3,15-10-2020,11:30:00,diltiazem hydrochloride,19.0,20.0,24.848485,15.424242,low
5,15-10-2020,11:30:00,ketamine hydrochloride,9.0,21.0,17.709677,11.935484,medium
6,15-10-2020,12:30:00,diltiazem hydrochloride,20.0,21.0,24.848485,15.424242,medium



So in general, we will be using the following format:

     > Series.str.function()

Series.str can be used to **access the values of the series as strings** and apply several methods to it.


Now suppose we want to form a new column based on the year of the experiments?

#### What can we do form a column containing the year?


In [ ]:
data_tidy['Date'].str.split('-')

0      [15, 10, 2020]
1      [15, 10, 2020]
2      [15, 10, 2020]
3      [15, 10, 2020]
4      [15, 10, 2020]
            ...      
103    [17, 10, 2020]
104    [17, 10, 2020]
105    [17, 10, 2020]
106    [17, 10, 2020]
107    [17, 10, 2020]
Name: Date, Length: 108, dtype: object

To extract the year we need to select the last element of each list

In [ ]:
data_tidy['Date'].str.split('-').apply(lambda x:x[2])

0      2020
1      2020
2      2020
3      2020
4      2020
       ... 
103    2020
104    2020
105    2020
106    2020
107    2020
Name: Date, Length: 108, dtype: object

But there are certain problems with this approach:

- The **dtype of the output is still an object**, we would prefer a number type
- The date format will always **not be in day-month-year**, it can vary 

Thus, to work with such date-time type of data, we can use a special method of pandas

## Datetime

Lets start with understanding a date-time type of data

#### How can we handle handle date-time data-types?

  - We can do using the `to_datetime()` function of pandas
  - It takes as input:
    - Array/Scalars with values having proper date/time format
    - `dayfirst`: Indicating if the day comes first in the date format used
    - `yearfirst`: Indicates if year comes first in the date format

Let's first merge our ```Date``` and ```time``` columns into a new timestamp column




In [ ]:
data_tidy['timestamp'] = data_tidy['Date']+ " "+ data_tidy['time']

In [ ]:
data_tidy.drop(['Date', 'time'], axis=1, inplace=True)

In [ ]:
data_tidy.head()

None,Drug_Name,Pressure,Temperature,Temperature_avg,Pressure_avg,temp_cat,timestamp
0,diltiazem hydrochloride,18.0,20.0,24.848485,15.424242,low,15-10-2020 10:30:00
1,docetaxel injection,26.0,23.0,30.387097,25.483871,medium,15-10-2020 10:30:00
2,ketamine hydrochloride,9.0,22.0,17.709677,11.935484,medium,15-10-2020 10:30:00
3,diltiazem hydrochloride,19.0,20.0,24.848485,15.424242,low,15-10-2020 11:30:00
4,docetaxel injection,29.0,25.0,30.387097,25.483871,medium,15-10-2020 11:30:00


Lets convert our `timestamp` col now

In [ ]:
data_tidy['timestamp'] = pd.to_datetime(data_tidy['timestamp']) # will leave to explore how you can mention datetime format by your own

data_tidy

None,Drug_Name,Pressure,Temperature,Temperature_avg,Pressure_avg,temp_cat,timestamp
0,diltiazem hydrochloride,18.0,20.0,24.848485,15.424242,low,2020-10-15 10:30:00
1,docetaxel injection,26.0,23.0,30.387097,25.483871,medium,2020-10-15 10:30:00
2,ketamine hydrochloride,9.0,22.0,17.709677,11.935484,medium,2020-10-15 10:30:00
3,diltiazem hydrochloride,19.0,20.0,24.848485,15.424242,low,2020-10-15 11:30:00
4,docetaxel injection,29.0,25.0,30.387097,25.483871,medium,2020-10-15 11:30:00
...,...,...,...,...,...,...,...
103,docetaxel injection,26.0,19.0,30.387097,25.483871,low,2020-10-17 08:30:00
104,ketamine hydrochloride,11.0,20.0,17.709677,11.935484,low,2020-10-17 08:30:00
105,diltiazem hydrochloride,9.0,13.0,24.848485,15.424242,low,2020-10-17 09:30:00
106,docetaxel injection,27.0,20.0,30.387097,25.483871,low,2020-10-17 09:30:00


In [ ]:
data_tidy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108 entries, 0 to 107
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Drug_Name        108 non-null    object        
 1   Pressure         108 non-null    float64       
 2   Temperature      108 non-null    float64       
 3   Temperature_avg  108 non-null    float64       
 4   Pressure_avg     108 non-null    float64       
 5   temp_cat         108 non-null    category      
 6   timestamp        108 non-null    datetime64[ns]
dtypes: category(1), datetime64[ns](1), float64(4), object(1)
memory usage: 10.3+ KB


The **type of `timestamp` column** has been **changed to `datetime`** from `object`

Now, Let's look at a single timestamp using Pandas

#### How can we **extract information** from a single **timestamp** using Pandas?

In [ ]:
ts = data_tidy['timestamp'][0]
ts

Timestamp('2020-10-15 10:30:00')

#### Now how can we extract the year from this date ?


In [ ]:
ts.year

2020

Similarly we can also access the month and day using the `month` and `day` attributes

In [ ]:
ts.month

10

In [ ]:
ts.day

15

#### But what if we want to know the name of the month or the day of the week on that date ?
  - We can find it using `month_name()` and `day_name()` methods

In [ ]:
ts.month_name()

'October'

In [ ]:
ts.day_name()

'Thursday'

In [ ]:
ts.dayofweek

3

In [ ]:
ts.hour

10

In [ ]:
ts.minute

30

... and so on

We can similarly extract minutes and seconds


#### This data parsing from string to date-time makes it easier to work with data

We can use this data from the columns as a whole using ```.dt``` object

In [ ]:
data_tidy['timestamp'].dt

- **`dt` gives properties of values in a column**

- From this **`DatetimeProperties` of column `'end'`**, we can **extract `year`**

In [ ]:
data_tidy['timestamp'].dt.year

0      2020
1      2020
2      2020
3      2020
4      2020
       ... 
103    2020
104    2020
105    2020
106    2020
107    2020
Name: timestamp, Length: 108, dtype: int64

Now, Let's **create the new column using these extracted values from the property**

We will use strfttime, short for stringformat time, to modify our datetime format

Let's learn this with the help of few examples

In [ ]:
data_tidy['timestamp'][0]

Timestamp('2020-10-15 10:30:00')

In [ ]:
print(data_tidy['timestamp'][0].strftime('%Y')) # Formatter for year

2020


In [ ]:
print(data_tidy['timestamp'][0].strftime('%m')) # Formatter for month

10


In [ ]:
print(data_tidy['timestamp'][0].strftime('%d')) # Formatter for day

15


In [ ]:
print(data_tidy['timestamp'][0].strftime('%H')) # Formatter for hour

10


In [ ]:
print(data_tidy['timestamp'][0].strftime('%M')) # Formatter for minutes

30


In [ ]:
print(data_tidy['timestamp'][0].strftime('%S')) # Formatter for seconds

00


Similarly we can combine the format types to modify the date-time format as per our convinience

In [ ]:
data_tidy['timestamp'][0].strftime('%m-%d')

'10-15'

## Writing to file


#### How can we write our dataframe to a csv file?


- We have to **provide the path and file_name** in which you want to store the data

In [ ]:
data_tidy.to_csv('pfizer_tidy.csv', sep=",") 